#imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import warnings
from IPython.display import display

#normalizar

In [ ]:
# Bases a normalizar
bases = [
    'VGG16_256_AVG.csv', 'VGG16_256_MAX.csv',
    'VGG19_128_AVG.csv', 'VGG19_128_MAX.csv',
    'VGG19_256_AVG.csv', 'VGG19_256_MAX.csv',
    'VGG16_256_AVG_PCA.csv', 'VGG16_256_MAX_PCA.csv',
    'VGG19_128_AVG_PCA.csv', 'VGG19_128_MAX_PCA.csv',
    'VGG19_256_AVG_PCA.csv', 'VGG19_256_MAX_PCA.csv'
]

for base in bases:
    print(f"🔄 Normalizando: {base}")
    df = pd.read_csv(base)

    # Separar atributos e classe
    X = df.drop(columns=['classe'])
    y = df['classe']

    # Normalizar
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Voltar para DataFrame e juntar
    df_scaled = pd.DataFrame(X_scaled, columns=X.columns)
    df_final = pd.concat([df_scaled, y], axis=1)

    # Salvar com novo nome
    nome_saida = base.replace('.csv', '_NORMALIZADA.csv')
    df_final.to_csv(nome_saida, index=False)

print("✅ Todas as bases foram normalizadas e salvas.")


🔄 Normalizando: VGG16_256_AVG.csv
🔄 Normalizando: VGG16_256_MAX.csv
🔄 Normalizando: VGG19_128_AVG.csv
🔄 Normalizando: VGG19_128_MAX.csv
🔄 Normalizando: VGG19_256_AVG.csv
🔄 Normalizando: VGG19_256_MAX.csv
🔄 Normalizando: VGG16_256_AVG_PCA.csv
🔄 Normalizando: VGG16_256_MAX_PCA.csv
🔄 Normalizando: VGG19_128_AVG_PCA.csv
🔄 Normalizando: VGG19_128_MAX_PCA.csv
🔄 Normalizando: VGG19_256_AVG_PCA.csv
🔄 Normalizando: VGG19_256_MAX_PCA.csv
✅ Todas as bases foram normalizadas e salvas.


#teste2

In [ ]:
# Ignorar avisos de convergência para uma saída mais limpa durante os testes
warnings.filterwarnings('ignore', category=Warning)

# --- 1. Definição das Funções de Teste ---

def carregar_base(caminho_arquivo):
    """Carrega uma base de dados a partir de um arquivo CSV."""
    df = pd.read_csv(caminho_arquivo)
    X = df.drop('classe', axis=1)
    y = df['classe']
    return X, y

DEFAULT_MAX_ITER = 1000

def testar_ativacoes(X, y):
    ativacoes = ['identity', 'logistic', 'tanh', 'relu']
    resultados = []
    print("   - Testando Funções de Ativação...")
    for func in ativacoes:
        modelo = make_pipeline(StandardScaler(), MLPClassifier(activation=func, random_state=1, max_iter=DEFAULT_MAX_ITER))
        score = cross_val_score(modelo, X, y, cv=10, scoring='accuracy')
        resultados.append({'activation': func, 'mean_acc': score.mean()*100})
    return pd.DataFrame(resultados).sort_values(by='mean_acc', ascending=False).iloc[0]

def testar_solvers(X, y, ativacao):
    solvers = ['sgd', 'adam']
    resultados = []
    print("   - Testando Solvers...")
    for solver in solvers:
        modelo = make_pipeline(StandardScaler(), MLPClassifier(activation=ativacao, solver=solver, random_state=1, max_iter=DEFAULT_MAX_ITER))
        score = cross_val_score(modelo, X, y, cv=10, scoring='accuracy')
        resultados.append({'solver': solver, 'mean_acc': score.mean()*100})
    return pd.DataFrame(resultados).sort_values(by='mean_acc', ascending=False).iloc[0]

# ==========================================================================================
# --- FUNÇÃO MODIFICADA ---
def testar_neuronios(X, y, ativacao, solver):
    """Testa a quantidade de neurônios usando a metodologia da aula."""

    # Calcula o número de atributos e classes do dataset atual
    num_atributos = X.shape[1]
    num_classes = len(np.unique(y))

    # Aplica as fórmulas para definir os valores a serem testados
    val_a = int((num_atributos + num_classes) / 2)
    val_i = num_atributos
    val_o = num_classes
    val_t = num_atributos + num_classes

    # Cria a lista de neurônios, remove duplicados e ordena
    neuronios_para_testar = sorted(list(set([val_a, val_i, val_o, val_t])))

    print(f"   - Testando Número de Neurônios (metodologia: {neuronios_para_testar})...")

    resultados = []
    for n in neuronios_para_testar:
        # Garante que n seja pelo menos 1
        if n == 0: continue
        modelo = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(n,), activation=ativacao, solver=solver, random_state=1, max_iter=DEFAULT_MAX_ITER))
        score = cross_val_score(modelo, X, y, cv=10, scoring='accuracy')
        resultados.append({'neurônios': n, 'mean_acc': score.mean()*100})

    if not resultados: # Caso todos os valores de neurônios sejam 0
        return pd.Series({'neurônios': num_atributos, 'mean_acc': 0})

    return pd.DataFrame(resultados).sort_values(by='mean_acc', ascending=False).iloc[0]
# --- FIM DA MODIFICAÇÃO ---
# ==========================================================================================


def testar_taxas(X, y, ativacao, solver, neur):
    neur = int(neur)
    taxas = [0.001, 0.01, 0.1]
    resultados = []
    print("   - Testando Taxas de Aprendizado...")
    for lr in taxas:
        modelo = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(neur,), activation=ativacao, solver=solver, learning_rate_init=lr, random_state=1, max_iter=DEFAULT_MAX_ITER))
        score = cross_val_score(modelo, X, y, cv=10, scoring='accuracy')
        resultados.append({'learning_rate': lr, 'mean_acc': score.mean()*100})
    return pd.DataFrame(resultados).sort_values(by='mean_acc', ascending=False).iloc[0]

def testar_iteracoes(X, y, ativacao, solver, neur, lr):
    neur = int(neur)
    iteracoes_teste = [500, 1000, 1500, 2000]
    resultados = []
    print("   - Testando Número de Iterações...")
    for i in iteracoes_teste:
        modelo = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(neur,), activation=ativacao, solver=solver, learning_rate_init=lr, max_iter=i, random_state=1))
        score = cross_val_score(modelo, X, y, cv=10, scoring='accuracy')
        resultados.append({'iteracoes': i, 'mean_acc': score.mean(), 'std': score.std()})
    return pd.DataFrame(resultados).sort_values(by='mean_acc', ascending=False).iloc[0]


#test

In [ ]:
# --- 2. Lista de Bases de Dados e Preparação do Loop ---
bases_normalizadas = [
    'VGG16_256_AVG_NORMALIZADA.csv', 'VGG16_256_MAX_NORMALIZADA.csv',
    'VGG19_128_AVG_NORMALIZADA.csv', 'VGG19_128_MAX_NORMALIZADA.csv',
    'VGG19_256_AVG_NORMALIZADA.csv', 'VGG19_256_MAX_NORMALIZADA.csv',
    'VGG16_256_AVG_PCA_NORMALIZADA.csv', 'VGG16_256_MAX_PCA_NORMALIZADA.csv',
    'VGG19_128_AVG_PCA_NORMALIZADA.csv', 'VGG19_128_MAX_PCA_NORMALIZADA.csv',
    'VGG19_256_AVG_PCA_NORMALIZADA.csv', 'VGG19_256_MAX_PCA_NORMALIZADA.csv'
]
resultados_finais = []

# --- 3. Execução do Loop Experimental ---
for base_csv in bases_normalizadas:
    print(f"\n🔎 Processando base: {base_csv}")
    try:
        X, y = carregar_base(base_csv)

        # Passos 1 a 4
        melhor_ativ = testar_ativacoes(X, y)
        ativ = melhor_ativ['activation']
        melhor_solver = testar_solvers(X, y, ativ)
        solver = melhor_solver['solver']
        melhor_neur = testar_neuronios(X, y, ativ, solver) # <-- Esta função agora usa a nova metodologia
        neur = melhor_neur['neurônios']
        melhor_lr = testar_taxas(X, y, ativ, solver, neur)
        lr = melhor_lr['learning_rate']

        # Passo 5
        resultado_final_cv = testar_iteracoes(X, y, ativ, solver, neur, lr)
        it = int(resultado_final_cv['iteracoes'])
        acc_final_cv = resultado_final_cv['mean_acc']
        std_final_cv = resultado_final_cv['std']

        # Teste 70/30 com a melhor configuração
        print("   - Rodando teste 70/30 com a melhor configuração...")
        config_final = {
            'hidden_layer_sizes': (int(neur),), 'activation': ativ, 'solver': solver,
            'learning_rate_init': lr, 'max_iter': it, 'random_state': 1
        }
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
        pipeline_holdout = make_pipeline(StandardScaler(), MLPClassifier(**config_final))
        pipeline_holdout.fit(X_train, y_train)
        y_pred = pipeline_holdout.predict(X_test)
        acc_holdout = accuracy_score(y_test, y_pred)

        # Exibir resultados no console
        print(f"\n✅ Melhor configuração encontrada para {base_csv}:")
        print(f"   Ativação={ativ}, Solver={solver}, Neurônios={int(neur)}, Taxa={lr}, Iterações={it}")
        print(f"   Acurácia Final (10-fold CV): {acc_final_cv*100:.2f}% (± {std_final_cv*100:.2f})")
        print(f"   Acurácia Final (70/30 Hold-Out): {acc_holdout*100:.2f}%")

        # Armazenar resultados
        resultados_finais.append({
            'Base de Dados': base_csv.replace('_NORMALIZADA.csv', ''),
            'Melhor Nº de Neurônios': int(neur),
            'Melhor Taxa de Aprendizado': lr,
            'Melhor Nº de Iterações': it,
            'Acurácia (10-fold CV)': f"{acc_final_cv*100:.2f}%",
            'Acurácia (70/30)': f"{acc_holdout*100:.2f}%"
        })

    except FileNotFoundError:
        print(f"❌ ERRO: Arquivo '{base_csv}' não encontrado. Pulando para o próximo.")
    except Exception as e:
        print(f"❌ ERRO: Ocorreu um problema ao processar '{base_csv}': {e}")


# --- 4. Geração e Salvamento da Tabela Final ---
if resultados_finais:
    tabela_df = pd.DataFrame(resultados_finais)
    print("\n\n--- Tabela Final com as Melhores Configurações e Comparações de Testes ---")
    display(tabela_df)

    nome_arquivo_saida = 'resultados_finais_mlp.csv'
    try:
        tabela_df.to_csv(nome_arquivo_saida, sep=';', encoding='utf-8-sig', index=False)
        print(f"\n✅ Tabela de resultados salva com sucesso no arquivo: '{nome_arquivo_saida}'")
    except Exception as e:
        print(f"\n❌ Ocorreu um erro ao salvar o arquivo CSV: {e}")

else:
    print("\nNenhum resultado foi gerado. Verifique os erros acima.")

#hiper 10 fold


In [ ]:
import pandas as pd
import numpy as np
import time
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from IPython.display import display, HTML

# Ignorar avisos para uma saída mais limpa
warnings.filterwarnings('ignore')

# --- 1. Armazenando as Melhores Configurações Encontradas (Seus Resultados) ---
best_configs = {
    'VGG16_256_AVG_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'sgd', 'hidden_layer_sizes': (258,), 'learning_rate_init': 0.01, 'max_iter': 500},
    'VGG16_256_MAX_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'sgd', 'hidden_layer_sizes': (516,), 'learning_rate_init': 0.001, 'max_iter': 500},
    'VGG19_128_AVG_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'sgd', 'hidden_layer_sizes': (258,), 'learning_rate_init': 0.001, 'max_iter': 500},
    'VGG19_128_MAX_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'sgd', 'hidden_layer_sizes': (512,), 'learning_rate_init': 0.001, 'max_iter': 1000},
    'VGG19_256_AVG_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'adam', 'hidden_layer_sizes': (4,), 'learning_rate_init': 0.001, 'max_iter': 500},
    'VGG19_256_MAX_NORMALIZADA.csv': {'activation': 'tanh', 'solver': 'adam', 'hidden_layer_sizes': (258,), 'learning_rate_init': 0.001, 'max_iter': 500},
    'VGG16_256_AVG_PCA_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'sgd', 'hidden_layer_sizes': (14,), 'learning_rate_init': 0.001, 'max_iter': 2000},
    'VGG16_256_MAX_PCA_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'sgd', 'hidden_layer_sizes': (14,), 'learning_rate_init': 0.001, 'max_iter': 1000},
    'VGG19_128_AVG_PCA_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'adam', 'hidden_layer_sizes': (4,), 'learning_rate_init': 0.001, 'max_iter': 1000},
    'VGG19_128_MAX_PCA_NORMALIZADA.csv': {'activation': 'identity', 'solver': 'sgd', 'hidden_layer_sizes': (4,), 'learning_rate_init': 0.001, 'max_iter': 1000},
    'VGG19_256_AVG_PCA_NORMALIZADA.csv': {'activation': 'relu', 'solver': 'sgd', 'hidden_layer_sizes': (7,), 'learning_rate_init': 0.1, 'max_iter': 500},
    'VGG19_256_MAX_PCA_NORMALIZADA.csv': {'activation': 'identity', 'solver': 'sgd', 'hidden_layer_sizes': (14,), 'learning_rate_init': 0.001, 'max_iter': 500}
}

# --- 2. Função de Teste Genérica ---
def run_test(X, y, config):
    pipeline = make_pipeline(StandardScaler(), MLPClassifier(random_state=1, **config))
    scores = cross_val_score(pipeline, X, y, cv=10, scoring='accuracy')
    return np.mean(scores)

# --- 3. Loop Principal para Análise de Sensibilidade ---
print("--- INICIANDO ANÁLISE DE SENSIBILIDADE DOS HIPERPARÂMETROS ---")

for base_name, best_config in best_configs.items():
    display(HTML(f"<h2>Análise para a Base: {base_name.replace('_NORMALIZADA.csv', '')}</h2>"))

    try:
        df = pd.read_csv(base_name)
        X = df.drop('classe', axis=1)
        y = df['classe']

        # --- a. Variando o Número de Neurônios ---
        current_results = []
        # Define a lista de neurônios a testar com base no nome do arquivo (PCA ou não)
        if 'PCA' in base_name:
            neuron_list = sorted(list(set([4, 7, 10, 14])))
        else:
            neuron_list = sorted(list(set([4, 258, 512, 516])))

        print(f"  -> Testando Neurônios: {neuron_list}...")
        for n_neurons in neuron_list:
            config = best_config.copy()
            config['hidden_layer_sizes'] = (n_neurons,)
            accuracy = run_test(X, y, config)
            current_results.append({'Neurônios': n_neurons, 'Acurácia (10-fold CV)': f"{accuracy*100:.2f}%"})

        display(HTML("<h4>a. Impacto do Número de Neurônios</h4>"))
        display(pd.DataFrame(current_results).set_index('Neurônios'))

        # --- b. Variando a Taxa de Aprendizado ---
        current_results = []
        learning_rate_list = [0.0001, 0.001, 0.01, 0.1]
        print(f"  -> Testando Taxas de Aprendizado: {learning_rate_list}...")
        for lr in learning_rate_list:
            config = best_config.copy()
            config['learning_rate_init'] = lr
            accuracy = run_test(X, y, config)
            current_results.append({'Taxa de Aprendizado': lr, 'Acurácia (10-fold CV)': f"{accuracy*100:.2f}%"})

        display(HTML("<h4>b. Impacto da Taxa de Aprendizado</h4>"))
        display(pd.DataFrame(current_results).set_index('Taxa de Aprendizado'))

        # --- c. Variando o Número de Iterações ---
        current_results = []
        max_iter_list = [200, 500, 1000, 1500, 2000]
        print(f"  -> Testando Número de Iterações: {max_iter_list}...")
        for n_iter in max_iter_list:
            config = best_config.copy()
            config['max_iter'] = n_iter
            accuracy = run_test(X, y, config)
            current_results.append({'Nº de Iterações': n_iter, 'Acurácia (10-fold CV)': f"{accuracy*100:.2f}%"})

        display(HTML("<h4>c. Impacto do Número de Iterações</h4>"))
        display(pd.DataFrame(current_results).set_index('Nº de Iterações'))

    except FileNotFoundError:
        print(f"!!!!!! ARQUIVO NÃO ENCONTRADO: {base_name}. Pulando. !!!!!!")
    except Exception as e:
        print(f"****** ERRO ao processar {base_name}: {e} ******")

print("\n--- ANÁLISE DE SENSIBILIDADE CONCLUÍDA ---")

--- INICIANDO ANÁLISE DE SENSIBILIDADE DOS HIPERPARÂMETROS ---


  -> Testando Neurônios: [4, 258, 512, 516]...


,Acurácia (10-fold CV)
Neurônios,
4,98.12%
258,98.87%
512,98.62%
516,98.62%


  -> Testando Taxas de Aprendizado: [0.0001, 0.001, 0.01, 0.1]...


,Acurácia (10-fold CV)
Taxa de Aprendizado,
0.0001,96.00%
0.0010,98.62%
0.0100,98.87%
0.1000,98.75%


...

  -> Testando Taxas de Aprendizado: [0.0001, 0.001, 0.01, 0.1]...


,Acurácia (10-fold CV)
Taxa de Aprendizado,
0.0001,95.87%
0.0010,98.50%
0.0100,98.00%
0.1000,97.62%


  -> Testando Número de Iterações: [200, 500, 1000, 1500, 2000]...


,Acurácia (10-fold CV)
Nº de Iterações,
200,98.25%
500,98.50%
1000,98.50%
1500,98.50%
2000,98.50%



--- ANÁLISE DE SENSIBILIDADE CONCLUÍDA ---


#hiper 70/30


In [ ]:
import pandas as pd
import numpy as np
import time
import warnings
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from IPython.display import display, HTML

# Ignorar avisos para uma saída mais limpa
warnings.filterwarnings('ignore')

# --- 1. Armazenando as Melhores Configurações Encontradas (Seus Resultados) ---
best_configs = {
    'VGG16_256_AVG_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'sgd', 'hidden_layer_sizes': (258,), 'learning_rate_init': 0.01, 'max_iter': 500},
    'VGG16_256_MAX_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'sgd', 'hidden_layer_sizes': (516,), 'learning_rate_init': 0.001, 'max_iter': 500},
    'VGG19_128_AVG_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'sgd', 'hidden_layer_sizes': (258,), 'learning_rate_init': 0.001, 'max_iter': 500},
    'VGG19_128_MAX_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'sgd', 'hidden_layer_sizes': (512,), 'learning_rate_init': 0.001, 'max_iter': 1000},
    'VGG19_256_AVG_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'adam', 'hidden_layer_sizes': (4,), 'learning_rate_init': 0.001, 'max_iter': 500},
    'VGG19_256_MAX_NORMALIZADA.csv': {'activation': 'tanh', 'solver': 'adam', 'hidden_layer_sizes': (258,), 'learning_rate_init': 0.001, 'max_iter': 500},
    'VGG16_256_AVG_PCA_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'sgd', 'hidden_layer_sizes': (14,), 'learning_rate_init': 0.001, 'max_iter': 2000},
    'VGG16_256_MAX_PCA_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'sgd', 'hidden_layer_sizes': (14,), 'learning_rate_init': 0.001, 'max_iter': 1000},
    'VGG19_128_AVG_PCA_NORMALIZADA.csv': {'activation': 'logistic', 'solver': 'adam', 'hidden_layer_sizes': (4,), 'learning_rate_init': 0.001, 'max_iter': 1000},
    'VGG19_128_MAX_PCA_NORMALIZADA.csv': {'activation': 'identity', 'solver': 'sgd', 'hidden_layer_sizes': (4,), 'learning_rate_init': 0.001, 'max_iter': 1000},
    'VGG19_256_AVG_PCA_NORMALIZADA.csv': {'activation': 'relu', 'solver': 'sgd', 'hidden_layer_sizes': (7,), 'learning_rate_init': 0.1, 'max_iter': 500},
    'VGG19_256_MAX_PCA_NORMALIZADA.csv': {'activation': 'identity', 'solver': 'sgd', 'hidden_layer_sizes': (14,), 'learning_rate_init': 0.001, 'max_iter': 500}
}

# --- 2. FUNÇÃO DE TESTE SIMPLIFICADA ---
def run_test_holdout(X, y, config):
    """Executa o teste com 70/30 Hold-Out."""
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    pipeline = make_pipeline(StandardScaler(), MLPClassifier(random_state=1, **config))
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return accuracy_score(y_test, y_pred)
# --- FIM DA ATUALIZAÇÃO ---


# --- 3. Loop Principal para Análise de Sensibilidade ---
print("--- INICIANDO ANÁLISE DE SENSIBILIDADE (APENAS TESTE 70/30) ---")

for base_name, best_config in best_configs.items():
    display(HTML(f"<h2>Análise para a Base: {base_name.replace('_NORMALIZADA.csv', '')}</h2>"))

    try:
        df = pd.read_csv(base_name)
        X = df.drop('classe', axis=1)
        y = df['classe']

        # --- a. Variando o Número de Neurônios ---
        current_results = []
        if 'PCA' in base_name:
            neuron_list = sorted(list(set([4, 7, 10, 14])))
        else:
            neuron_list = sorted(list(set([4, 258, 512, 516])))

        print(f"  -> Testando Neurônios: {neuron_list}...")
        for n_neurons in neuron_list:
            config = best_config.copy()
            config['hidden_layer_sizes'] = (n_neurons,)
            # --- ALTERAÇÃO: Rodar apenas o teste 70/30 ---
            acc_holdout = run_test_holdout(X, y, config)
            current_results.append({
                'Neurônios': n_neurons,
                'Acurácia (70/30)': f"{acc_holdout*100:.2f}%"
            })

        display(HTML("<h4>a. Impacto do Número de Neurônios</h4>"))
        display(pd.DataFrame(current_results).set_index('Neurônios'))

        # --- b. Variando a Taxa de Aprendizado ---
        current_results = []
        learning_rate_list = [0.001, 0.01, 0.1]
        print(f"  -> Testando Taxas de Aprendizado: {learning_rate_list}...")
        for lr in learning_rate_list:
            config = best_config.copy()
            config['learning_rate_init'] = lr
            # --- ALTERAÇÃO: Rodar apenas o teste 70/30 ---
            acc_holdout = run_test_holdout(X, y, config)
            current_results.append({
                'Taxa de Aprendizado': lr,
                'Acurácia (70/30)': f"{acc_holdout*100:.2f}%"
            })

        display(HTML("<h4>b. Impacto da Taxa de Aprendizado</h4>"))
        display(pd.DataFrame(current_results).set_index('Taxa de Aprendizado'))

        # --- c. Variando o Número de Iterações ---
        current_results = []
        max_iter_list = [200, 500, 1000, 1500, 2000]
        print(f"  -> Testando Número de Iterações: {max_iter_list}...")
        for n_iter in max_iter_list:
            config = best_config.copy()
            config['max_iter'] = n_iter
            # --- ALTERAÇÃO: Rodar apenas o teste 70/30 ---
            acc_holdout = run_test_holdout(X, y, config)
            current_results.append({
                'Nº de Iterações': n_iter,
                'Acurácia (70/30)': f"{acc_holdout*100:.2f}%"
            })

        display(HTML("<h4>c. Impacto do Número de Iterações</h4>"))
        display(pd.DataFrame(current_results).set_index('Nº de Iterações'))

    except FileNotFoundError:
        print(f"!!!!!! ARQUIVO NÃO ENCONTRADO: {base_name}. Pulando. !!!!!!")
    except Exception as e:
        print(f"****** ERRO ao processar {base_name}: {e} ******")

print("\n--- ANÁLISE DE SENSIBILIDADE (70/30) CONCLUÍDA ---")

--- INICIANDO ANÁLISE DE SENSIBILIDADE (APENAS TESTE 70/30) ---


  -> Testando Neurônios: [4, 258, 512, 516]...


,Acurácia (70/30)
Neurônios,
4,97.50%
258,98.75%
512,98.75%
516,98.75%


  -> Testando Taxas de Aprendizado: [0.001, 0.01, 0.1]...


,Acurácia (70/30)
Taxa de Aprendizado,
0.001,98.75%
0.010,98.75%
0.100,99.17%


...

  -> Testando Taxas de Aprendizado: [0.001, 0.01, 0.1]...


,Acurácia (70/30)
Taxa de Aprendizado,
0.001,97.92%
0.010,97.08%
0.100,96.67%


  -> Testando Número de Iterações: [200, 500, 1000, 1500, 2000]...


,Acurácia (70/30)
Nº de Iterações,
200,97.92%
500,97.92%
1000,97.92%
1500,97.92%
2000,97.92%



--- ANÁLISE DE SENSIBILIDADE (70/30) CONCLUÍDA ---
